In [1]:
#!/usr/bin/python3
import numpy as np
import transforms3d


import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
import csv
import os

from matplotlib import cm

# Append the root dir
import sys, roslib, os
lias_anp_dir = roslib.packages.get_pkg_dir('lias_anp')
scripts_dir = os.path.abspath(os.path.join(lias_anp_dir, 'scripts'))
sys.path.append(scripts_dir)
from utils.sonar_data_processor import SonarDataReader
from utils.match_pairs import get_match_pairs
from anp.anp_alg import AnPAlgorithm
from tri.tri import ANRS, GTRS, gradient_descent
from utils.pose2matrix import pose_to_transform_matrix, ros_pose_to_transform_matrix, transform_matrix_to_ros_pose
from utils.coordinate_system_transform import coordinate_transform_Pose, coordinate_transform_Pose_back, coordinate_transform_pt, coordinate_transform_pt_back
from utils.transformation_matrix_add_noise import add_noise_to_pose

import yaml
yaml_file_path = os.path.join(lias_anp_dir, 'yaml/odom.yaml')
with open(yaml_file_path, 'r') as file:
    params = yaml.safe_load(file)
    RECONSTRUCTION_ERROR_THRESHOLD = params['RECONSTRUCTION_ERROR_THRESHOLD']
    RECORD = params['RECORD']
    DATA_PATH = params['data_path']
    ANP_METHOD = params['ANP_METHOD']

sonar_data_dir = str(lias_anp_dir) + DATA_PATH
reord_dir = str(lias_anp_dir) + "/record/" + ANP_METHOD
reader = SonarDataReader(filepath = sonar_data_dir)
reader.read_data()
data = reader.get_data()

sonar_data_dir

'/home/clp/catkin_ws/src/lias_anp/data/heading_in_same_direction/Rho_noise0.00001theta_noise0.00001/eight_all_noisy/sonar_data_noisy.csv'

In [2]:
from functools import reduce
def intersect_multiple_arrays(array_list):
    array_lists = [list(arr) for arr in array_list]
    return list(reduce(np.intersect1d, array_lists))

def get_match_pairs(theta_Rhos_across_times, pts_indices_across_times):
    common_indices = intersect_multiple_arrays(pts_indices_across_times)
    matched_theta_Rho_across_times = [] 
    for pts_indicei, theta_Rhoi in zip(pts_indices_across_times, theta_Rhos_across_times):
        matched_theta_Rho_i = theta_Rhoi[[np.where(pts_indicei == idx)[0][0] for idx in common_indices]]
        matched_theta_Rho_across_times.append(matched_theta_Rho_i)
    return matched_theta_Rho_across_times, common_indices

theta_Rhos_across_times = []
pts_indices_across_times = []
T_tri_accross_times = []

INITIALIZE_TIMES = 5
for i in range(INITIALIZE_TIMES):
    Ti_tri = coordinate_transform_Pose(ros_pose_to_transform_matrix(data[i]['pose']))
    theta_Rhoi = data[i]['si_q_theta_Rho']
    pts_indicei = data[i]['pts_indice']
    T_tri_accross_times.append(Ti_tri)
    theta_Rhos_across_times.append(theta_Rhoi)
    pts_indices_across_times.append(pts_indicei)

    
matched_theta_Rho_across_times, common_indices = get_match_pairs(theta_Rhos_across_times, pts_indices_across_times)
points_num = len(common_indices)
points_num
# Now we have T_tri_accross_times and matched_theta_Rho_across_times

32

In [3]:
# We need to iterate through points
matched_theta_Rho_across_times = np.array(matched_theta_Rho_across_times)
# for each points
for point_index in range(points_num):
    theta_Rhos = matched_theta_Rho_across_times[:,point_index,:]
    
    A_list = []
    b_list = []

    for i in range(INITIALIZE_TIMES-1):
        T0 = T_tri_accross_times[i]
        T1 = T_tri_accross_times[i+1]
        T_matrix = np.linalg.inv(T1) @ T0
        theta_Rhos = matched_theta_Rho_across_times[i]
        theta_Rho_primes = matched_theta_Rho_across_times[i+1]
            
        # 将线性方程组写成矩阵形式 A @ P = B
        R_matrix = T_matrix[:3, :3]
        t = T_matrix[:3, 3]
        r1 = R_matrix[0, :]
        r2 = R_matrix[1, :]

        for theta_Rho, theta_Rho_prime in zip(theta_Rhos, theta_Rho_primes):
            theta = -theta_Rho[0]
            theta_prime = -theta_Rho_prime[0]
            R = theta_Rho[1]  # example value for R
            R_prime = theta_Rho_prime[1] # example value for R'
            
            a1 = np.array([-1, np.tan(theta), 0])
            b1 = 0 
            a2 = np.tan(theta_prime) * r2 - r1
            b2 = t[0] - np.tan(theta_prime) * t[1]
            a3 = t.T @ R_matrix
            b3 = (R_prime**2 - R**2 - np.linalg.norm(t)**2) / 2

            A = np.vstack([a1, a2, a3])
            A = A @ np.linalg.inv(T0)
            b = np.array([b1, b2, b3])
            
            # 将 A 和 b 添加到列表中
            A_list.append(A)
            b_list.append(b)
            
        # 循环结束后，将列表转换为 NumPy 数组
        A_stacked = np.stack(A_list, axis=0)
        b_stacked = np.stack(b_list, axis=0)
        
        # 打印结果的形状
        print("A_stacked shape:", A_stacked.shape)
        print("b_stacked shape:", b_stacked.shape)

    x, residuals, rank, s = np.linalg.lstsq(A_stacked, b_stacked, rcond=None)
    print(x)




ValueError: matmul: Input operand 1 has a mismatch in its core dimension 0, with gufunc signature (n?,k),(k,m?)->(n?,m?) (size 4 is different from 3)